In [3]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image

In [ ]:
import os
import numpy as np
from PIL import Image

folder_path = r'D:\Project Files\ADNI'
images = []
common_size = (256, 256)
for root, dirs, files in os.walk(folder_path):  
    for file_name in files:
        if file_name.lower().endswith('.png'): 
            image_path = os.path.join(root, file_name)
            try:
                image = Image.open(image_path)
                image = image.resize(common_size) 
                image_data = np.array(image)
                images.append(image_data)
                print(f"Loaded {file_name} with shape {image_data.shape} from {root}")
            except Exception as e:
                print(f"Could not load {file_name}: {e}")

images = np.array(images)
print(f"Total images loaded: {len(images)}")


In [ ]:
images = []
labels = []

for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.lower().endswith('.png'):
            image_path = os.path.join(root, file_name)
            try:
                image = Image.open(image_path)
                image = image.resize(common_size) 
                image_data = np.array(image)
                images.append(image_data)  
                label = os.path.basename(root) 
                labels.append(label) 
                
                print(f"Loaded {file_name} with shape {image_data.shape} from {root}, label: {label}")
            except Exception as e:
                print(f"Could not load {file_name}: {e}")

images = np.array(images)
labels = np.array(labels)
print(f"Total images loaded: {len(images)}, Total labels: {len(labels)}")


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

print(f"Encoded labels: {labels_encoded}")
print(f"Categorical labels: {labels_categorical}")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)
X_train = X_train / 255.0
X_test = X_test / 255.0

print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  

model.add(Dense(len(np.unique(labels_encoded)), activation='softmax'))  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

In [ ]:
print(X_train.shape)

In [10]:
# If your images are grayscale
X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)  # Add channels dimension

# If your images are RGB
# X_train = X_train.reshape(X_train.shape[0], 256, 256, 3)  # Uncomment if images are RGB


In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))  
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) 
model.add(Dense(len(np.unique(labels_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

In [12]:
# Reshape images to include the channel dimension (for grayscale images)
X_train = X_train.reshape(-1, 256, 256, 1)
X_test = X_test.reshape(-1, 256, 256, 1)


In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

print(f"Predicted classes: {predicted_classes}")
print(f"True classes: {true_classes}")
